In [101]:
import time
import pandas as pd

from bs4 import BeautifulSoup
from twscrape.logger import set_log_level
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

FIN

In [127]:
# set up chromedriver and direct to link
driver_path = r"/home/jasmine/chromedriver-linux64/chromedriver"
service = Service(executable_path=driver_path)
driver = webdriver.Chrome(service=service)
options = Options()
options.add_argument("window-size=1200x600")
link = "https://cns.utexas.edu/academics/undergraduate-study/all-undergraduate-programs"
driver.get(link)

In [128]:
# select most recent catalog (arbitrary)
dropdown = driver.find_element(By.ID, "year")
select = Select(dropdown)
select.select_by_index(1)

In [129]:
ALL_DATA = []

# there are still pages left to display...
btn = driver.find_elements(By.XPATH, "//*[contains(text(), 'Next')]")
displayed = btn[0].get_attribute("aria-disabled") == "true"

while displayed == False: 
    
    btn[0].click()
    displayed = btn[0].get_attribute("aria-disabled") == "true"

    # get info on page
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    degrees_ls = soup.select('ul[aria-labelledby="Undergraduate Degrees List"] > li')

    for degree in degrees_ls:
        # get each col from the table
        degree_type = degree.select_one('div[aria-label="Type"]').text.strip()
        program_name = degree.select_one('div[aria-label="Name"]').text.strip()
        year_of_entry = degree.select_one('div[aria-label="Year of Entry"]').text.strip()
        
        # grab links
        requirements_link = degree.select_one('a[title="Download checklist"]')
        what_to_take_link = degree.select_one('a[title="4-Year Plan"]')
        requirements_url = requirements_link['href'] if requirements_link else "N/A"
        what_to_take_url = what_to_take_link['href'] if what_to_take_link else "N/A"
        
        ALL_DATA.append({
            "type": degree_type,
            "program": program_name,
            "catalog": year_of_entry,
            "reqs_url": requirements_url,
            "4_yr_plan": what_to_take_url,
        })

In [130]:
ALL_DATA

[{'type': 'BS',
  'program': 'Biology: Option 2',
  'catalog': '2022–2024',
  'reqs_url': 'https://cns.utexas.edu/sites/default/files/uploads/degree-checklists/ut-bs-biology-option2-degree-checklist-22-24-catalog.pdf',
  '4_yr_plan': 'https://cns.utexas.edu/sites/default/files/uploads/degree-plans/ut-bs-biology-option2-4-yr-plan-22-24-catalog_0.pdf'},
 {'type': 'BS',
  'program': 'Biology: Option 3',
  'catalog': '2022–2024',
  'reqs_url': 'https://cns.utexas.edu/sites/default/files/uploads/degree-checklists/ut-bs-biology-option3-degree-checklist-22-24-catalog.pdf',
  '4_yr_plan': 'https://cns.utexas.edu/sites/default/files/uploads/degree-plans/ut-bs-biology-option3-4-yr-plan-22-24-catalog_0.pdf'},
 {'type': 'BS',
  'program': 'Biology: Option 4',
  'catalog': '2022–2024',
  'reqs_url': 'https://cns.utexas.edu/sites/default/files/uploads/degree-checklists/ut-bs-biology-option4-degree-checklist-22-24-catalog.pdf',
  '4_yr_plan': 'https://cns.utexas.edu/sites/default/files/uploads/degree

In [131]:
len(ALL_DATA)

83

In [132]:
DATA = pd.DataFrame(ALL_DATA)
DATA

,type,program,catalog,reqs_url,4_yr_plan
0,BS,Biology: Option 2,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
1,BS,Biology: Option 3,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
2,BS,Biology: Option 4,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
3,BS,Biology: Option 5,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
4,BS,Biology: Option 7,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
...,...,...,...,...,...
78,C,Scientific Computation,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,N/A
79,BS,Statistics and Data Sciences,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
80,BS,Textiles & Apparel,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,https://cns.utexas.edu/sites/default/files/upl...
81,C,UTeach-Natural Sciences Accelerated Track,2022–2024,https://cns.utexas.edu/sites/default/files/upl...,N/A


In [133]:
DATA.to_csv("CNS_DEGREE_INFO.csv", index=False)